In [1]:
import pandas as pd
from pathlib import Path
import jsonlines

In [2]:
model = 'gpt-4o-mini'
dist_folder = 'no-distil-test'
filename = 'batch_gpt4omini_init_assessment-gpt4o-mini-distill_fWzHuiddTOgVL0jUnIhIGsEQ_output_.jsonl'
# batch_gpt4omini_init_assessment-gpt4o-mini-distill_fWzHuiddTOgVL0jUnIhIGsEQ_output_


In [3]:
basepath = (Path() / '..' /'..'/ 'logs' / model / dist_folder).resolve()

In [14]:
df = pd.read_csv(basepath / 'prompts.csv')

In [15]:
df['model_answer'] = ''
with jsonlines.open(basepath / filename) as reader:
    for obj in reader:
        i = int(obj['custom_id'].removeprefix('init_assessment-gpt4o-mini-distill_'))
        df.loc[i, 'model_answer'] = obj['response']['body']['choices'][0]['message']['content']

In [16]:
df['model_answer'][3]

'540 meters.'

In [22]:
dol = 0
spac = 0
noex = 0
noans = 0
perc = 0
ids = []
for i, a in df['model_answer'].items():
    if not a:
        noans += 1
    elif '$' in a.lower():
        dol += 1
    elif ' ' in a.lower():
        spac += 1
    elif '%' in a:
        perc += 1
    else:
        noex += 1
        ids.append(i)

print(f'Empty string: {noans}')
print(f'Dollar: {dol}')
print(f'Space: {spac}')
print(f'Percent: {perc}')
print(f'No extraction: {noex}')
ids[:10]

Empty string: 0
Dollar: 361
Space: 880
Percent: 13
No extraction: 65


[37, 66, 127, 136, 165, 193, 233, 253, 263, 269]

In [65]:
df['model_ex1'] = df['model_answer']
for i,row in df.iterrows():
    if '$' in row['model_answer']:
        df.loc[i, 'model_ex1'] = row['model_answer'].replace('$', '')
    if '%' in df['model_ex1'][i]:
        df.loc[i, 'model_ex1'] = df['model_ex1'][i].replace('%','')
    if ',' in df['model_ex1'][i]:
        df.loc[i, 'model_ex1'] = df['model_ex1'][i].replace(',','')
    if ' ' in df['model_ex1'][i]:
        vls = df['model_ex1'][i].split(' ')
        vs = []
        for v in vls:
            try:
                vs.append(float(v))
                continue
            except:
                pass
        if len(vs) == 1:
            df.loc[i, 'model_ex1'] = vs[0]
        
    else:
        try:
            df.loc[i, 'model_ex1'] = float(df['model_ex1'][i])
        except:
            df.loc[i, 'model_ex1'] = df['model_ex1'][i]
    df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])



In [60]:
def save_df(df : pd.DataFrame, model : str, dist_folder : str, edition : str):
    base_path = Path()
    base_path = base_path / '..' / 'logs' / model / dist_folder / f'{edition}.csv'
    df.to_csv(base_path)

In [66]:

for i, row in df[df['model_ex1_t'] != float].iterrows():
    inp = input(f'{i} : What is the extracted answer : {row["model_ex1"]}.     True: {row['extracted_answer']}. Question: {row['question']}')
    try:
        df.loc[i, 'model_ex1'] = float(inp)
    except:
        print(f'{i} could not be parsed')


In [67]:
for i, row in df.iterrows():
    df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])
    

In [69]:
df['correct'] = df['model_ex1'] == df['extracted_answer']

In [70]:
df['correct'].value_counts()

correct
False    925
True     394
Name: count, dtype: int64

In [58]:
for i, row in df[df['correct'] == False][0:10].iterrows():
    print(f'TRUE: {row['extracted_answer']}, MODEL: {row['model_answer']}')

TRUE: 70000.0, MODEL: $50,000.
TRUE: 20.0, MODEL: Wendi needs to give her chickens 30 cups of feed in the final meal of the day.
TRUE: 64.0, MODEL: $82.00
TRUE: 260.0, MODEL: Toulouse, Charleston, and Seattle have a total of 96 sheep.
TRUE: 160.0, MODEL: The answer is: 220 minutes.
TRUE: 45.0, MODEL: He is 80 miles from home at the end of those 4 hours.
TRUE: 366.0, MODEL: 120 downloads.
TRUE: 694.0, MODEL: The total cost was $1,124.
TRUE: 13.0, MODEL: 5 years.
TRUE: 18.0, MODEL: Melanie started with 21 vacuum cleaners.
